In [1]:
%pip install numpy pandas seaborn matplotlib plotly scikit-learn nltk wordcloud pyspellchecker beautifulsoup4 prettytable

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Loading of Data

In [2]:
import pandas as pd
df = pd.read_csv(r'C:\Users\tiled\OneDrive\Desktop\DataSet\IMDB-Dataset.csv',encoding='latin-1')

In [3]:
df.head(10)["Reviews"]

0    *Disclaimer: I only watched this movie as a co...
1    I am writing this in hopes that this gets put ...
2    Really, I could write a scathing review of thi...
3    If you saw the other previous spoof movies by ...
4    This movie I saw a day early for free and I st...
5    Honestly, what is wrong with you, Hollywood? N...
6    I was given a free ticket to this film; so I c...
7    OK, so "Disastrous" isn't an imaginative barb ...
8    Jason Friedberg and Aaron Seltzer, the way eve...
9    Honestly the worst movie ever made. Theatre fu...
Name: Reviews, dtype: object

### Customizing Stopwords for Enhanced Text Preprocessing

*Enhancing the default stopword list by adding domain-specific words and removing negations to improve text analysis accuracy.*

In [4]:
from nltk.corpus import stopwords
import nltk

stop_words = stopwords.words('english')
nltk.download('averaged_perceptron_tagger')

new_stopwords = [
"would", "shall", "could", "might", "movie", "movies", "film", "films",
"cinema", "director", "directors", "actor", "actors", "actress", "actresses",
"cast", "screenplay", "plot", "story", "character", "characters", "scene",
"scenes", "soundtrack", "soundtracks","sequel", "prequel", "adaptation", "trailer", "genre", "genres", "release",
"theaters", "theatre", "performance", "performances", "acting", "blockbuster",
"indie", "oscars", "oscar", "award-winning", "box", "office", "premiere",
"rating", "ratings", "reviews", "animation", "cinematography", "editing", "script", "narrative",
"direction", "score", "soundtrack"
]

stop_words.extend(new_stopwords)
stop_words.remove("not")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\tiled\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [5]:
stop_words=set(stop_words)
print(len(stop_words))

233


### Data Cleaning and Text Preprocessing Pipeline

*Comprehensive functions for cleaning and preparing text data, including removing special characters, URLs, stopwords, and expanding contractions to enhance the quality of the dataset.*

### Correction of Typos

*Written text often contains errors, such as “Fen” instead of “Fan.” To rectify these errors, a dictionary is employed to map words to their correct forms based on similarity.*

In [6]:
from spellchecker import SpellChecker

spell = SpellChecker()

def correct_typos(content):
    """
    Parameters:
    - content (str): The input text to be corrected.

    Returns:
    - str: The text with corrected typos.
    """
    try:
        words = content.split()
        corrected_words = []
        misspelled = spell.unknown(words)

        for word in words:
            if word in misspelled:
                corrected_word = spell.correction(word)
                corrected_words.append(corrected_word)
            else:
                corrected_words.append(word)
        
        corrected_content = ' '.join(corrected_words)
        return corrected_content
    except Exception as e:
        print(f"Error in correcting typos: {e}")
        return content

### Mapping and Replacement

*This involves mapping words to standardized language equivalents. For instance, words like “b4” and “ttyl,” commonly understood by humans as “before” and “talk to you later,” pose challenges for machines. Normalization entails mapping such words to their standardized counterparts.*

In [7]:
import re

def load_mapping_dictionary():
    """
    Returns:
    - dict: A dictionary mapping non-standard words to standardized words.
    """
    mapping_dict = {
        "b4": "before",
        "ttyl": "talk to you later",
        "u": "you",
        "r": "are",
        "lol": "laughing out loud",
        "idk": "i do not know",
        "btw": "by the way",
        "omg": "oh my god",
        "imo": "in my opinion",
        "np": "no problem",
        "gr8": "great",
        "l8r": "later",
        "gtg": "got to go",
        "thx": "thanks",
        "pls": "please",
        "plz": "please",
        "bc": "because",
        "cuz": "because",
        "y'all": "you all",
        "luv": "love",
        "wanna": "want to",
        "gonna": "going to",
        "hafta": "have to",
        "kinda": "kind of",
        "sorta": "sort of",
        "gimme": "give me",
        "lemme": "let me",
        "whatcha": "what are you",
        "whaddaya": "what do you",
    }
    return mapping_dict

def normalize_text(content, mapping_dict):
    """
    Parameters:
    - content (str): The input text to be normalized.
    - mapping_dict (dict): A dictionary mapping non-standard words to standardized words.
    
    Returns:
    - str: The normalized text.
    """
    try:
        words = content.split()
        normalized_words = []
        
        for word in words:
            clean_word = re.sub(r'[^\w\s]', '', word.lower())
            if clean_word in mapping_dict:
                replacement = mapping_dict[clean_word]
                normalized_words.append(replacement)
            else:
                normalized_words.append(word)
        
        normalized_content = ' '.join(normalized_words)
        return normalized_content
    except Exception as e:
        print(f"Error in normalizing text: {e}")
        return content

### Expanding Contractions

*Convert contractions like “don’t” to “do not” or “I’ll” to “I will.”*

In [8]:
contractions = {
        r"won't": "will not",
        r"can't": "cannot",
        r"i'm": "i am",
        r"ain't": "is not",
        r"let's": "let us",
        r"ma'am": "madam",
        r"shan't": "shall not",
        r"n't": " not",
        r"'re": " are",
        r"'s": " is",
        r"'d": " would",
        r"'ll": " will",
        r"'ve": " have",
        r"'m": " am",
        r"he's": "he is",
        r"she's": "she is",
        r"it's": "it is",
        r"that's": "that is",
        r"there's": "there is",
        r"who's": "who is",
        r"what's": "what is",
        r"where's": "where is",
        r"when's": "when is",
        r"why's": "why is",
        r"how's": "how is",
        r"would've": "would have",
        r"could've": "could have",
        r"should've": "should have",
        r"might've": "might have",
        r"must've": "must have",
        r"wouldn't": "would not",
        r"couldn't": "could not",
        r"shouldn't": "should not",
        r"mightn't": "might not",
        r"mustn't": "must not",
        r"don't": "do not",
        r"doesn't": "does not",
        r"didn't": "did not",
        r"hasn't": "has not",
        r"haven't": "have not",
        r"hadn't": "had not",
        r"can't've": "cannot have",
        r"shan't've": "shall not have",
        r"wouldn't've": "would not have",
        r"couldn't've": "could not have",
        r"shouldn't've": "should not have",
        r"mightn't've": "might not have",
        r"mustn't've": "must not have",
        r"i'd": "i would",
        r"i'll": "i will",
        r"i've": "i have",
        r"i'm": "i am",
        r"you'd": "you would",
        r"you'll": "you will",
        r"you've": "you have",
        r"you're": "you are",
        r"he'd": "he would",
        r"he'll": "he will",
        r"he's": "he is",
        r"she'd": "she would",
        r"she'll": "she will",
        r"she's": "she is",
        r"it'd": "it would",
        r"it'll": "it will",
        r"it's": "it is",
        r"they'd": "they would",
        r"they'll": "they will",
        r"they're": "they are",
        r"we'd": "we would",
        r"we'll": "we will",
        r"we're": "we are",
        r"there'd": "there would",
        r"that'd": "that would",
        r"who'd": "who would",
        r"who'll": "who will",
        r"who're": "who are",
        r"what've": "what have",
        r"where've": "where have",
        r"when've": "when have",
        r"why've": "why have",
        r"how've": "how have",
        r"lets": "let us",
        r"lets'": "let us",
        r"gonna": "going to",
        r"wanna": "want to",
        r"gotta": "got to",
        r"oughtn't": "ought not",
        r"needn't": "need not",
        r"daren't": "dare not",
        r"maam": "madam",
        r"gimme": "give me",
        r"lemme": "let me",
        r"whatcha": "what are you",
        r"whaddaya": "what do you",
        r"gimme": "give me",
        r"gonna": "going to",
        r"gotta": "got to",
        r"hafta": "have to",
        r"wanna": "want to",
        r"ain't": "am not",
        r"y'all": "you all",
        r"coulda": "could have",
        r"woulda": "would have",
        r"shoulda": "should have",
        r"'bout": "about",
        r"'til": "until",
        r"can't've": "cannot have",
        r"could've": "could have",
        r"might've": "might have",
        r"must've": "must have",
        r"should've": "should have",
        r"would've": "would have",
        r"you'd've": "you would have",
    }

def contraction_expansion(content):
    """
    Parameters:
    - content (str): The input text.

    Returns:
    - str: Text with expanded contractions.
    """
    
    for contraction, expanded in contractions.items():
        pattern = re.compile(contraction, flags=re.IGNORECASE)
        content = pattern.sub(expanded, content)
    return content

### Removing Accents and Diacritics
*Sometimes, people use accented characters like é, ö, etc. to signify emphasis on a particular letter during pronunciation. Normalize text by removing accents and diacritical marks from characters.*

In [9]:
import unicodedata

def remove_accents(content):
    """
    Parameters:
    - content (str): The input text to be normalized.
    
    Returns:
    - str: The text with accents and diacritics removed.
    """
    try:
        normalized = unicodedata.normalize('NFKD', content)
        without_accents = ''.join([c for c in normalized if not unicodedata.combining(c)])
        return without_accents
    except Exception as e:
        print(f"Error in removing accents: {e}")
        return content

### Removing Extra Whitespace

*Normalize text by removing extra spaces and leading/trailing spaces*

In [10]:
def remove_extra_whitespace(text):
    """
    Parameters:
    - text (str): The input text to be normalized.
    
    Returns:
    - str: The text with extra spaces removed.
    """
    return ' '.join(text.split())

### Eliminating HTML Tags

*In cases where raw text originates from sources such as web scraping or screen capture, it often carries along HTML tags. These tags introduce unwanted noise and contribute little to the comprehension and analysis of the text. Therefore, it becomes necessary to strip them.*

In [11]:
from bs4 import BeautifulSoup

def remove_html_tags(text):
    """
    Parameters:
    - text (str): The input text containing HTML tags.
    
    Returns:
    - str: The text with HTML tags removed.
    """
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

### Handling URLs

*Frequently, individuals include URLs, particularly in social media content, to supplement context with additional information. However, URLs tend to vary across samples and can be considered noise.*

In [12]:
import re 

def remove_url(text):
    return re.sub(r"(https|http)?:\S*", "", text)

### Case Standardising and Removing Special Characters

*Special characters are non-alphanumeric characters. The characters like %,$,&, etc are special. In most NLP tasks, these characters add no value to text understanding and induce noise into algorithms. We can use regular expressions to remove special characters.*

In [13]:
def remove_special_character(content):
    content = content.lower()
    return re.sub(r'\W+',' ', content)

### Removing Stopwords

*In most cases, stopwords like I, am, me, etc. don’t add any information that can help in modeling. Keeping them in the text introduces unnecessary noise and can significantly increase the dimensionality of feature vectors, which can negatively impact both computation cost and model accuracy.*

In [14]:
def remove_stopwords(content):
    clean_data = []
    for i in content.split():
        if i.strip().lower() not in stop_words and i.strip().lower().isalpha():
            clean_data.append(i.strip().lower())
    return " ".join(clean_data)

In [15]:
def data_cleaning(content):
    # content = correct_typos(content)
    mapping_dict = load_mapping_dictionary()
    content = remove_extra_whitespace(content)
    content = normalize_text(content, mapping_dict)
    content = contraction_expansion(content)
    content = remove_url(content)
    content = remove_accents(content)
    # content = remove_html_tags(content)
    content = remove_special_character(content)
    content = remove_stopwords(content)
    return content

### Applying Data Cleaning to Reviews and Displaying Results

In [16]:
%%time
pd.options.display.max_colwidth = 1000
df['Reviews_clean']=df['Reviews'].apply(data_cleaning)
df.head(5)

CPU times: total: 2min 57s
Wall time: 3min 27s


,Ratings,Reviews,Movies,Resenhas,Reviews_clean
0,1.0,"*Disclaimer: I only watched this movie as a conditional agreement. And I see films for free. I wouldn't be caught dead giving my hard earned money to these idiots.Well, to explain the depth of this 'film', I could write my shortest review, ever. Don't see this movie. It is by far the stupidest, lamest, most lazy, and unbelievably UNFUNNY movie I have ever seen. It is a total disaster. But since my hatred for this movie, and the others like it, extends far beyond one viewing, I think I'll go on for a bit.I don't know any of the people in the movie besides Carmen Electra, Vanessa Minnillo, and Kim Kardashian, but it doesn't matter. They're all horrible, though I think that was the point. The editing is flat out horrible, and possibly blatant continuity errors make this crapfast even crappier than I thought it would be. Now I know that these films are not supposed to be serious at all, but come on, it's film-making 101 that if someone gets a minor facial cut, it should be there in the...",Disaster Movie,"* IsenÃ§Ã£o de responsabilidade: eu sÃ³ assisti esse filme como um acordo condicional. E eu vejo filmes de graÃ§a. Eu nÃ£o seria pego morto dando meu dinheiro suado a esses idiotas. Bem, para explicar a profundidade desse 'filme', eu poderia escrever minha crÃ­tica mais curta de todos os tempos. NÃ£o vÃª este filme. Ã de longe o filme mais estÃºpido, lamenta, preguiÃ§oso e inacreditavelmente UNFUNNY que eu jÃ¡ vi. Ã um desastre total. Mas como o meu Ã³dio por este filme e por outros, se estende muito alÃ©m de uma exibiÃ§Ã£o, acho que vou continuar um pouco. NÃ£o conheÃ§o nenhuma das pessoas do filme alÃ©m de Carmen Electra, Vanessa Minnillo, e Kim Kardashian, mas isso nÃ£o importa. Eles sÃ£o todos horrÃ­veis, embora eu ache que esse seja o ponto. A ediÃ§Ã£o Ã© horrÃ­vel e, possivelmente, erros de continuidade flagrantes tornam essa porcaria ainda mais horrÃ­vel do que eu pensava. Agora eu sei que esses filmes nÃ£o devem ser sÃ©rios, mas vamos lÃ¡, Ã© o cinema 101 que se alguÃ©m f...",disclaimer watched conditional agreement see free not caught dead giving hard earned money idiots well explain depth write shortest review ever not see far stupidest lamest lazy unbelievably unfunny ever seen total disaster since hatred others like extends far beyond one viewing think go bit not know people besides carmen electra vanessa minnillo kim kardashian not matter horrible though think point flat horrible possibly blatant continuity errors make crapfast even crappier thought know not supposed serious come making someone gets minor facial cut next shot someone gets cut sword blood least cut though since narnia get away give disaster pass jokes thoughtless mindless physical gags obviously take popular last year late well including best picture nominees know saddest thing stupid not care much money make many cameos sorry ass excuses taking away jobs writers truly deserve attention lionsgate thought better taste ashamed making kind crap jason friedberg aaron seltzer burn hell g...
1,1.0,"I am writing this in hopes that this gets put over the previous review of this ""film"". How anyone can find this slop entertaining is completely beyond me. First of all a spoof film entitled ""Disaster Movie"", should indeed be a spoof on disaster films. Now I have seen 1 (yes count them, 1) disaster film being spoofed, that being ""Twister"". How does Juno, Iron Man, Batman, The Hulk, Alvin and the Chipmunks, Amy Winehouse, or Hancock register as Disaster films? Selzterwater and Failburg once again have shown that they lack any sort of writing skill and humor. Having unfortunately been tortured with Date Movie and Epic Movie I know exactly what to expect from these two...no plot, no jokes just bad references and cheaply remade scenes from other films. Someone should have informed them that satire is more than just copy and paste from one film to another, though I shouldn't say that because some of these ac

## Feature Engineering

In [17]:
df['Label'] = df['Ratings'].apply(lambda x: '1' if x >= 7 else ('0' if x<=4 else '2'))

df=df[df.Label<'2']
data=df[['Reviews_clean','Reviews','Ratings','Label']]

data['Label'].value_counts()

Label
0    60000
1    60000
Name: count, dtype: int64

### Lemmatization

In [18]:
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize  

nltk.download('wordnet')
nltk.download('punkt')

class LemmaTokenizer(object):
    def __init__(self):
        self.wordnetlemma = WordNetLemmatizer()
    
    def __call__(self, reviews):
        """
        Lemmatizes a list of tokens to their base form.

        Parameters:
        - tokens (list): A list of words to be lemmatized.

        Returns:
        - list: A list of lemmatized words.
        """
        return [self.wordnetlemma.lemmatize(word) for word in word_tokenize(reviews)]

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tiled\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tiled\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Text Vectorization with CountVectorizer and TF-IDF

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [20]:
train_data, test_data = train_test_split(data, test_size=0.3, random_state=42, shuffle=True)

In [21]:
y_train = train_data['Label']
y_test = test_data['Label']

In [22]:
test_data.columns

Index(['Reviews_clean', 'Reviews', 'Ratings', 'Label'], dtype='object')

In [23]:
y_test_list=y_test.tolist()
test_list=test_data['Reviews_clean'].tolist()
rating_list=test_data['Ratings'].tolist()

### Model Evaluation

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [25]:
def vectorize_text(train_data, test_data, tokenizer, ngram_range=(1, 1), min_df=10, max_features=500):
    """
    Vectorizes text data using specified vectorizer type.

    Parameters:
    - train_data (pd.Series): The training text data to be vectorized.
    - test_data (pd.Series): The test text data to be vectorized.
    - tokenizer (callable): A tokenizer function or class instance.
    - ngram_range (tuple): The range of n-grams to consider.
    - min_df (int): Minimum document frequency for terms.
    - max_features (int): Maximum number of features to consider.

    Returns:
    - np.ndarray: The vectorized training text data.
    - np.ndarray: The vectorized test text data.
    - Vectorizer: The fitted vectorizer instance (TfidfVectorizer).
    """
  
    vectorizer = TfidfVectorizer(analyzer="word", 
                                 tokenizer=tokenizer, 
                                 ngram_range=ngram_range, 
                                 min_df=min_df,
                                 max_features=max_features)
  

    x_train_vectorized = vectorizer.fit_transform(train_data).toarray()
    x_test_vectorized = vectorizer.transform(test_data).toarray()

    return x_train_vectorized, x_test_vectorized, vectorizer

In [26]:
x_train_ngrams_tfidf_vectorized, x_test_ngrams_tfidf_vectorized, ngram_tfidf_vectorizer = vectorize_text(
    train_data['Reviews_clean'], 
    test_data['Reviews_clean'], 
    tokenizer=LemmaTokenizer(), 
    ngram_range=(1, 3), 
    min_df=10, 
    max_features=10000
)

c:\Users\tiled\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


**Define a Logistic Regression model with specific hyperparameters**

In [27]:
basic_logistic_regression = LogisticRegression(
    penalty='l2',
    dual=False,
    tol=0.0001,
    C=10,
    solver='lbfgs',
    max_iter=200,
    multi_class='auto',
    verbose=0,
    warm_start=False,
    n_jobs=None
)

**Define a pipeline with TF-IDF vectorization and Logistic Regression**

In [28]:
tfidf_logistic_pipeline = Pipeline(
    steps=[
        ('vect', TfidfVectorizer(
            analyzer="word",
            tokenizer=LemmaTokenizer(),
            ngram_range=(1, 3),
            min_df=10,
            max_features=10000
        )),
        ('classifier', LogisticRegression(
            penalty='l2',
            dual=False,
            tol=0.0001,
            C=10,
            solver='lbfgs',
            max_iter=100,
            multi_class='auto',
            verbose=0,
            warm_start=False,
            n_jobs=None
        ))
])

In [29]:
%%time
basic_logistic_regression.fit(x_train_ngrams_tfidf_vectorized,y_train)

CPU times: total: 12min 46s
Wall time: 2min 18s


c:\Users\tiled\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=10, max_iter=200)

In [30]:
%%time
tfidf_logistic_pipeline.fit(train_data['Reviews_clean'],y_train)

c:\Users\tiled\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CPU times: total: 1min 2s
Wall time: 2min 39s


c:\Users\tiled\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('vect',
                 TfidfVectorizer(max_features=10000, min_df=10,
                                 ngram_range=(1, 3),
                                 tokenizer=<__main__.LemmaTokenizer object at 0x0000026D2BEC9640>)),
                ('classifier', LogisticRegression(C=10))])

In [31]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score, accuracy_score , confusion_matrix

**Evaluate the basic Logistic Regression model**

In [32]:
precision_logistic = precision_score(y_test, basic_logistic_regression.predict(x_test_ngrams_tfidf_vectorized), average='micro')
recall_logistic = recall_score(y_test, basic_logistic_regression.predict(x_test_ngrams_tfidf_vectorized), average='micro')
auc_logistic = roc_auc_score(y_test, basic_logistic_regression.predict_proba(x_test_ngrams_tfidf_vectorized)[:, 1], multi_class='ovo', average='macro')
f1_logistic = f1_score(y_test, basic_logistic_regression.predict(x_test_ngrams_tfidf_vectorized), average="weighted")
accuracy_logistic = accuracy_score(y_test, basic_logistic_regression.predict(x_test_ngrams_tfidf_vectorized))

print("Precision Score for Logistic Regression: {:.2f}".format(precision_logistic))
print("Recall Score for Logistic Regression: {:.2f}".format(recall_logistic))
print("AUC Score for Logistic Regression: {:.2f}".format(auc_logistic))
print("F1 Score for Logistic Regression: {:.2f}".format(f1_logistic))
print("Accuracy Score for Logistic Regression: {:.2f}".format(accuracy_logistic))

Precision Score for Logistic Regression: 0.89
Recall Score for Logistic Regression: 0.89
AUC Score for Logistic Regression: 0.96
F1 Score for Logistic Regression: 0.89
Accuracy Score for Logistic Regression: 0.89


**Evaluate the TF-IDF Logistic Regression Pipeline**

In [33]:
precision_pipeline = precision_score(y_test, tfidf_logistic_pipeline.predict(test_data['Reviews_clean']), average='micro')
recall_pipeline = recall_score(y_test, tfidf_logistic_pipeline.predict(test_data['Reviews_clean']), average='micro')
auc_pipeline = roc_auc_score(y_test, tfidf_logistic_pipeline.predict_proba(test_data['Reviews_clean'])[:, 1], multi_class='ovo', average='macro')
f1_pipeline = f1_score(y_test, tfidf_logistic_pipeline.predict(test_data['Reviews_clean']), average="weighted")
accuracy_pipeline = accuracy_score(y_test, tfidf_logistic_pipeline.predict(test_data['Reviews_clean']))

print("Precision Score for Logistic Regression Pipeline: {:.2f}".format(precision_pipeline))
print("Recall Score for Logistic Regression Pipeline: {:.2f}".format(recall_pipeline))
print("AUC Score for Logistic Regression Pipeline: {:.2f}".format(auc_pipeline))
print("F1 Score for Logistic Regression Pipeline: {:.2f}".format(f1_pipeline))
print("Accuracy Score for Logistic Regression Pipeline: {:.2f}".format(accuracy_pipeline))

Precision Score for Logistic Regression Pipeline: 0.89
Recall Score for Logistic Regression Pipeline: 0.89
AUC Score for Logistic Regression Pipeline: 0.96
F1 Score for Logistic Regression Pipeline: 0.89
Accuracy Score for Logistic Regression Pipeline: 0.89


***Interpret and print the results***

In [40]:
def interpret_sentiment(output):
    return "Positive" if output == "1" else "Negative"

In [47]:
def evaluate_sentiment(review_text, real_sentiment, model):
    """
    Evaluates the sentiment of a given review text using the provided model and compares it to the real sentiment.

    Parameters:
    - review_text (str): The text of the review to evaluate.
    - real_sentiment (str): The actual sentiment of the review ("Positive" or "Negative").
    - model: The trained sentiment analysis model.

    Returns:
    - None: Prints the real sentiment, predicted sentiment, and whether the prediction was correct.
    """
   
    output = model.predict([review_text])

    predicted_sentiment = interpret_sentiment(output[0])

    print(f"Real Sentiment: {real_sentiment}")
    print(f"Predicted Sentiment: {predicted_sentiment}")

    if real_sentiment == predicted_sentiment:
        print("The model predicted the sentiment correctly.")
    else:
        print("The model did not predict the sentiment correctly.")

### Testing the Model with IMDB Movie Reviews

In [51]:
review_text = (
    "Beautiful is not a word that I enjoy using for a film with such a muted color pallet. "
    "However, for Robert Eggers' Nosferatu, after viewing it in its entirety, it is the only word that comes to mind. "
    "Eggers has constructed this film meticulously, with a precision seen in very few directors these days. "
    "Heck, more so than any previous effort he's put to screen. You can tell he enjoys working with a bigger budget "
    "and he knows what to do with it as well. Lavish, gothic set design and costumes across the board. "
    "Wonderful special FX, both CG and practical makeup. The cinematography was insane. Sweeping camera shots and "
    "uncomfortable closeups, all framed in a 35mm letterbox format. It was a spectacle worth the wait, especially on "
    "the IMAX screen I saw it on. The acting was phenomenal. Lily Rose-Depp should win awards for her all-out performance. "
    "Both horrific and touching all at the same time. Willem Dafoe provided some much-needed spice for the second half. "
    "Without his character, the film would've slipped a star as the story starts getting a little long-winded towards the end. "
    "My only gripe with the cast was Bill Skarsgård as Count Orlok. I felt he could've played into the character's sensitivity "
    "a bit more. He was cold-hearted, yes, but he did still have love in his heart. The story, adapted from both Dracula and "
    "Nosferatu... but mostly Nosferatu, was very well written. With dialog that felt natural and authentic to the time period, "
    "another one of Eggers' specialties, the film flowed rather nicely and almost transports you to that time and place. "
    "Impressively, when the film does eventually start getting bogged down a bit, Eggers makes sure to pick things right back "
    "up again to never keep the audience bored, and with a film like this that could've happened easily. Overall, a pretty great "
    "adaptation of a classic story. Equal parts horror and mystery, and a bittersweet ending that almost made me cry. I loved it, "
    "and a nice addition to Robert Eggers' growing filmography."
)

real_sentiment = "Positive"

evaluate_sentiment(review_text, real_sentiment, tfidf_logistic_pipeline)

Real Sentiment: Positive
Predicted Sentiment: Positive
The model predicted the sentiment correctly.


In [50]:
new_review_text = (
    "Poor content, poor backdrops, scenes without a meaning, predictable outcomes and a plot that is a recycled loop. "
    "If it's gonna be one of the first series you will ever see it might look good. Otherwise naaaaah!"
)

real_sentiment = "Negative"

evaluate_sentiment(review_text, real_sentiment, tfidf_logistic_pipeline)

Real Sentiment: Negative
Predicted Sentiment: Negative
The model predicted the sentiment correctly.


In [49]:
review_text = (
    "Story is good. Not a fan of one of the characters getting engaged with an 8 year old girl who he marries at 11 after she gets her period. "
    "Very disturbing and pulls focus away attention from the rest of the story that is interesting on its own without the pedophilia. "
    "While understand it was a different time, it clearly was frowned upon and very creepy. I'm curious why no one questioned why he would be interested in a little girl instead of a grown woman. "
    "I'm episode 6 and waiting for the supernatural aspect to kick in which at this point isn't much. Hopefully the last episodes redeem the show but I doubt it."
)

real_sentiment = "Negative"

evaluate_sentiment(review_text, real_sentiment, tfidf_logistic_pipeline)

Real Sentiment: Negative
Predicted Sentiment: Negative
The model predicted the sentiment correctly.


In [48]:
review_text = (
    "Adapting One Hundred Years of Solitude into a cinematic work appears to be an extremely difficult task for two main reasons. "
    "First, this novel belongs to a specific literary movement, pioneered by South American writers, particularly Gabriel García Márquez, "
    "where the story is narrated through the blending of reality and fantasy. At times, this fusion is so intense that separating the two seems impossible. "
    "Naturally, translating such scenes into cinema risks making them appear absurd and failing to achieve a satisfactory cinematic form. "
    "However, watching the first episode of the series One Hundred Years of Solitude revealed that the creators managed to convey this magical and surreal feeling "
    "to the audience without making it seem ridiculous. "
    "The second reason is the inherent complexity of the novel. One Hundred Years of Solitude is challenging to read due to its repeated use of identical names for different characters, "
    "as well as its non-linear narrative and frequent disruptions of the timeline. These elements may exhaust the reader. "
    "Fortunately, such issues are absent in the series, which has successfully transformed the non-linear narrative into a linear one, allowing it to establish a strong connection with the audience."
)

real_sentiment = "Positive"

evaluate_sentiment(review_text, real_sentiment, tfidf_logistic_pipeline)

Real Sentiment: Positive
Predicted Sentiment: Negative
The model did not predict the sentiment correctly.


### Future Directions for Improving Sentiment Analysis

*Issue and Cause*

- **Issue**: The model struggles to accurately predict sentiment for short reviews, despite achieving high accuracy (89%) on longer reviews from the IMDB dataset.

- **Cause**:
  - **Training Data Bias**: The model is trained primarily on longer reviews, which provide more context and complex patterns for sentiment analysis.
  - **Polarity and Sarcasm**: Words like "like" and "hate" may appear in both positive and negative contexts, especially in longer reviews, making it difficult for the model to determine sentiment in short reviews.
  - **Negation Handling**: Short reviews may lack sufficient context for the model to correctly interpret negations or sarcasm.

## Solutions

1. **Augment Training Data**:
   - Add more short reviews to the training dataset, ensuring a balanced representation of positive and negative sentiments. This helps the model learn to recognize sentiment in brief statements.

2. **Use Pre-trained Embeddings**:
   - Incorporate pre-trained word embeddings like GloVe or Word2Vec to provide richer semantic information for individual words, aiding the model in understanding sentiment in short reviews.

3. **Fine-tune the Model**:
   - Fine-tune the model on a dataset that includes a variety of review lengths, including short ones, to improve its adaptability to different contexts.

4. **Feature Engineering**:
   - Add features that capture sentiment more directly, such as sentiment lexicons or part-of-speech tags, to help the model better understand short reviews.

5. **Model Complexity**:
   - Experiment with more complex models, such as neural networks, which might capture nuances in short reviews better than simpler models.

6. **Hyperparameter Tuning**:
   - Adjust hyperparameters to improve the model's ability to handle short reviews, potentially increasing its sensitivity to context and sentiment cues.

In [45]:
short_reviews = [
    "I like this movie.",
    "I hate this movie.",
    "This movie is great!",
    "This movie is terrible.",
    "Amazing film!",
    "Awful film."
]

for review in short_reviews:
    output = tfidf_logistic_pipeline.predict([review])
    print(f"Review: '{review}' - Sentiment: {interpret_sentiment(output[0])}")

Review: 'I like this movie.' - Sentiment: Negative
Review: 'I hate this movie.' - Sentiment: Positive
Review: 'This movie is great!' - Sentiment: Positive
Review: 'This movie is terrible.' - Sentiment: Negative
Review: 'Amazing film!' - Sentiment: Positive
Review: 'Awful film.' - Sentiment: Negative
